## 훈련과정 : R·파이썬 기반 빅데이터 분석 전문가 양성과정
- 교과목 평가 : 머신러닝 기반 데이터 분석
- 성명 : 위정훈
- 점수 :

### Q. 아래사항을 준수하여 타이타닉 생존자 예측모델을 개발하세요.

- 데이터 : 
  - 제공 데이터 파일 : titanic3.csv
  - 훈련/검증용 데이터 : 평가 데이터 = 8 : 2
  - 훈련/검증용 데이터로 모델 학습 및 검증하고 평가 데이터는 최종 평가에만 사용


- 모델 개발 방법 :
  - 데이터 전처리 및 탐색적 분석을 통하여 파생변수 최소 2개 이상 개발
  - 알고리즘은 최소한 3개 이상 적용(Decision Tree, Random Forest, Logistic Regression은 필수)


- 훈련 및 평가 방법 :
  - GridSearchCV API를 활용하여 교차검증 및 최적 하이퍼파라미터 찾아서 학습 및 검증 수행

# import libraries and set korean font

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm  # 폰트 관련 용도

나눔 폰트 설치

In [0]:
!apt-get update -qq
!apt-get install fonts-nanum* -qq

Selecting previously unselected package fonts-nanum.
(Reading database ... 133872 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [0]:
sys_font=fm.findSystemFonts()
print(f"sys_font number: {len(sys_font)}")

nanum_font = [f for f in sys_font if 'Nanum' in f]
print(f"nanum_font number: {len(nanum_font)}")

nanum_font

sys_font number: 48
nanum_font number: 31


['/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf',
 '/usr/share/fonts/truetype/nanum/NanumMyeongjoEcoBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumBrush.ttf',
 '/usr/share/fonts/truetype/nanum/NanumBarunpenR.ttf',
 '/usr/share/fonts/truetype/nanum/NanumMyeongjoBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicEcoBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareRoundL.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicExtraBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareR.ttf',
 '/usr/share/fonts/truetype/nanum/NanumMyeongjoEcoExtraBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareRoundEB.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareB.ttf',
 '/usr/share/fonts/truetype/nanum/NanumMyeongjoEco.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothic.ttf',
 '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf',
 '/usr/share/fonts/truetype/

In [0]:
path =  '/usr/share/fonts/truetype/nanum/NanumBarunGothicLight.ttf'  
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)
plt.rcParams["font.family"] = font_name
matplotlib.rcParams['axes.unicode_minus'] = False

fm._rebuild()

NanumBarunGothic


# import data file

In [20]:
tt = pd.read_csv('titanic3.csv')
display (tt.sample(5))

print ('\n',tt.info())

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
484,2,1,"Lemore, Mrs. (Amelia Milley)",female,34.0,0,0,C.A. 34260,10.50,F33,S,14,NaN,"Chicago, IL"
274,1,1,"Spedden, Mr. Frederic Oakley",male,45.0,1,1,16966,134.50,E34,C,3,NaN,"Tuxedo Park, NY"
432,2,1,"Harris, Mr. George",male,62.0,0,0,S.W./PP 752,10.50,NaN,S,15,NaN,London
1171,3,0,"Sage, Master. William Henry",male,14.5,8,2,CA. 2343,69.55,NaN,S,NaN,67.0,NaN
655,3,0,"Backstrom, Mr. Karl Alfred",male,32.0,1,0,3101278,15.85,NaN,S,D,NaN,"Ruotsinphytaa, Finland New York, NY"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
pclass       1309 non-null int64
survived     1309 non-null int64
name         1309 non-null object
sex          1309 non-null object
age          1046 non-null float64
sibsp        1309 non-null int64
parch        1309 non-null int64
ticket       1309 non-null object
fare         1308 non-null float64
cabin        295 non-null object
embarked     1307 non-null object
boat         486 non-null object
body         121 non-null float64
home.dest    745 non-null object
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB

 None


# Preprocessing

## drop unnecessary columns

In [21]:
tt.drop(['name','ticket','embarked','body','home.dest',], axis=1, inplace=True)

display (tt.sample(5))

,pclass,survived,sex,age,sibsp,parch,fare,cabin,boat
955,3,0,female,NaN,3,1,25.4667,NaN,NaN
691,3,0,male,35.0,0,0,8.0500,NaN,NaN
623,3,0,female,6.0,4,2,31.2750,NaN,NaN
310,1,0,male,57.0,1,1,164.8667,NaN,NaN
1280,3,0,male,22.0,0,0,7.8958,NaN,NaN


### null data formatting

 - `isnull().sum()` 함수 통해 null 값 있는 columns 확인
 - `unique()` 함수 통해 data들의 형태 확인

In [23]:
print (tt.isnull().sum())
print ( f'{tt.age.unique()}\n\n{tt.fare.unique()}\n\n{tt.cabin.unique()}\n\n{tt.boat.unique()}')
print ( f'{tt.age.nunique()}\n\n{tt.fare.nunique()}\n\n{tt.cabin.nunique()}\n\n{tt.boat.nunique()}')

pclass         0
survived       0
sex            0
age          263
sibsp          0
parch          0
fare           1
cabin       1014
boat         823
dtype: int64
[29.    0.92  2.   30.   25.   48.   63.   39.   53.   71.   47.   18.
 24.   26.   80.     nan 50.   32.   36.   37.   42.   19.   35.   28.
 45.   40.   58.   22.   41.   44.   59.   60.   33.   17.   11.   14.
 49.   76.   46.   27.   64.   55.   70.   38.   51.   31.    4.   54.
 23.   43.   52.   16.   32.5  21.   15.   65.   28.5  45.5  56.   13.
 61.   34.    6.   57.   62.   67.    1.   12.   20.    0.83  8.    0.67
  7.    3.   36.5  18.5   5.   66.    9.    0.75 70.5  22.5   0.33  0.17
 40.5  10.   23.5  34.5  20.5  30.5  55.5  38.5  14.5  24.5  60.5  74.
  0.42 11.5  26.5 ]

[211.3375 151.55    26.55    77.9583   0.      51.4792  49.5042 227.525
  69.3     78.85    30.      25.925  247.5208  76.2917  75.2417  52.5542
 221.7792  26.      91.0792 135.6333  35.5     31.     164.8667 262.375
  55.      30.5     50.4

age column에서 현재 age column(null data 포함)과 age_fix column(null data를 `fillna()` 적용한 column)으로 분리

pclass 와 sex 에 따라 age의 평균값과 최빈값이 다름을 확인

In [34]:
print (tt.age.mean(), '\t',tt.age.median(), '\n')
print (tt['age'][tt['pclass']==3].count(), tt['age'][tt['pclass']==3].mean(), tt['age'][tt['pclass']==3].mode())
print (tt['age'][tt['pclass']==2].count(), tt['age'][tt['pclass']==2].mean(), tt['age'][tt['pclass']==2].mode())
print (tt['age'][tt['pclass']==1].count(), tt['age'][tt['pclass']==1].mean(), tt['age'][tt['pclass']==1].mode())
print (tt['age'][tt['sex']=='female'].count(), tt['age'][tt['sex']=='female'].mean(), tt['age'][tt['sex']=='female'].mode())
print (tt['age'][tt['sex']=='male'].count(), tt['age'][tt['sex']=='male'].mean(), tt['age'][tt['sex']=='male'].mode())

29.881137667304014 	 28.0 

501 24.81636726546906 0    22.0
dtype: float64
261 29.506704980842912 0    24.0
dtype: float64
284 39.15992957746479 0    36.0
dtype: float64
388 28.68708762886598 0    22.0
dtype: float64
658 30.58522796352583 0    21.0
dtype: float64


- 전체 age 데이터
- 형제+배우자가 동승하지 않은 경우 (sibsp == 0)
- 부모+자녀가 동승하지 않은 경우 (parch == 0)
- 형제+배우자가 동승한 경우 (sibsp > 0)
- 부모+자녀가 동승한 경우 (parch > 0)
- 부모자녀 또는 형제배우자 동승자 없는 경우
- 부모자녀 또는 형제배우자 동승자 있는 경우

위 경우들에서 mean, median, mode 값이 모두 다름 확인,

특히 부모자녀 동승자 있는 경우 미성년자 비율이 많아 mode가 2.0으로 극단적으로 낮아지는 경우 발생

In [39]:
# print (tt.sibsp.unique())
# print (tt.parch.unique())

print ('age 전체 데이터\n', tt['age'].describe(), tt['age'].mode() )
print ('부모자녀 동승자 없는 경우\n' , tt['age'][ tt['parch']==0 ].describe(), tt['age'][ tt['parch']==0 ].mode(), '\n',)
print ('형제배우자 동승자 없는 경우\n' ,tt['age'][ tt['sibsp']==0 ].describe(), tt['age'][ tt['sibsp']==0 ].mode(), '\n',)
print ('부모자녀 동승자 있는 경우\n' , tt['age'][ tt['parch'] > 0 ].describe(), tt['age'][ tt['parch'] > 0 ].mode(), '\n',)
print ('형제배우자 동승자 있는 경우\n' ,tt['age'][ tt['sibsp'] > 0 ].describe(), tt['age'][ tt['sibsp'] > 0 ].mode(), '\n',)
print ('부모자녀나 형제배우자 동승자 없는 경우\n' , tt['age'][(tt['parch']==0) | (tt['sibsp']==0) ].describe(), tt['age'][(tt['parch']==0) | (tt['sibsp']==0) ].mode(), '\n',)
print ('부모자녀나 형제배우자 동승자 있는 경우\n' , tt['age'][((tt['parch'] > 0) | (tt['sibsp'] > 0))].describe(), tt['age'][((tt['parch'] > 0) | (tt['sibsp'] > 0))].mode(), '\n',)

age 전체 데이터
 count    1046.000000
mean       29.881138
std        14.413493
min         0.170000
25%        21.000000
50%        28.000000
75%        39.000000
max        80.000000
Name: age, dtype: float64 0    24.0
dtype: float64
부모자녀 동승자 없는 경우
 count    768.000000
mean      31.969401
std       12.422617
min        5.000000
25%       22.000000
50%       29.000000
75%       39.000000
max       80.000000
Name: age, dtype: float64 0    24.0
1    30.0
dtype: float64 

형제배우자 동승자 없는 경우
 count    685.000000
mean      30.921766
std       13.059751
min        0.330000
25%       22.000000
50%       28.000000
75%       38.000000
max       80.000000
Name: age, dtype: float64 0    22.0
dtype: float64 

부모자녀 동승자 있는 경우
 count    278.000000
mean      24.112122
std       17.633611
min        0.170000
25%        8.000000
50%       22.000000
75%       38.000000
max       70.000000
Name: age, dtype: float64 0    2.0
dtype: float64 

형제배우자 동승자 있는 경우
 count    361.000000
mean      27.906537
std       16.52

age가 null data 이면서, parch가 있는 case는 총 29개

In [49]:
display (tt[tt['age'].isnull()][tt.parch > 0].values.tolist() , )
print ('\n', tt[tt['age'].isnull()][tt.parch > 0].count() ,'\n', )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


[[1, 1, 'female', nan, 0, 1, 55.0, 'E33', '6'],
 [3, 0, 'female', nan, 0, 2, 15.2458, nan, nan],
 [3, 0, 'female', nan, 0, 2, 7.75, nan, nan],
 [3, 0, 'male', nan, 1, 2, 23.45, nan, nan],
 [3, 0, 'female', nan, 1, 2, 23.45, nan, nan],
 [3, 0, 'male', nan, 1, 2, 23.45, nan, nan],
 [3, 0, 'female', nan, 1, 2, 23.45, nan, nan],
 [3, 0, 'male', nan, 3, 1, 25.4667, nan, nan],
 [3, 0, 'female', nan, 3, 1, 25.4667, nan, nan],
 [3, 0, 'female', nan, 3, 1, 25.4667, nan, nan],
 [3, 0, 'female', nan, 3, 1, 25.4667, nan, nan],
 [3, 0, 'female', nan, 0, 4, 25.4667, nan, nan],
 [3, 1, 'male', nan, 1, 1, 15.2458, nan, 'C'],
 [3, 1, 'male', nan, 1, 1, 15.2458, nan, 'C'],
 [3, 1, 'female', nan, 0, 2, 15.2458, nan, 'C'],
 [3, 1, 'male', nan, 1, 1, 22.3583, nan, 'C'],
 [3, 1, 'female', nan, 1, 1, 22.3583, 'F E69', 'D'],
 [3, 1, 'female', nan, 0, 2, 22.3583, nan, 'D'],
 [3, 0, 'male', nan, 8, 2, 69.55, nan, nan],
 [3, 0, 'female', nan, 8, 2, 69.55, nan, nan],
 [3, 0, 'female', nan, 8, 2, 69.55, nan, nan],


 pclass      29
survived    29
sex         29
age          0
sibsp       29
parch       29
fare        29
cabin        2
boat         7
dtype: int64 



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [0]:
print (tt['age'][(tt.parch.notnull()) & (tt.sibsp.notnull())].unique())

fare columns의 null data에는 해당 pclass의 평균값 입력

In [0]:
print (tt[tt['fare'].isnull()])
print (tt[ tt['fare'].isnull() ]['pclass'].max())
# print (tt['fare'][tt['pclass']==3].mean())
print (tt['fare'][tt['pclass']==tt[ tt['fare'].isnull() ]['pclass'].max()].mean())

In [0]:
tt.fare.fillna(value = (tt['fare'][tt['pclass']==tt[ tt['fare'].isnull() ]['pclass'].max()].mean()), inplace=True)
print (tt.fare.isnull().sum())

### fit_transforming

In [0]:
# tt.cabin = tt['cabin'].str[:1]

# tt.cabin = LabelEncoder().fit_transform( tt.cabin )
display(tt.cabin)